In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [6]:
# ! pip install -U sentence-transformers

In [14]:
# df = pd.read_csv('dataset/text_distance/dataset.csv')
df = pd.read_csv('dataset/semantic_similarity/dataset.csv')

In [31]:
# df

In [68]:
df['Key'].unique()

array(['ธัญพืช', 'Cereals',
       'ครั่ง รวมทั้งกัม เรซิน น้ำเลี้ยง(แซบ) และสิ่งสกัดอื่นๆ จากพืช',
       'Lac ; gum, resins and other vegetable saps and extracts', nan,
       'ไขมันและน้ำมันที่ได้จากสัตว์หรือพืช และผลิตภัณฑ์ที่แยกได้จากไขมันและน้ำมันดังกล่าว ไขมันที่บริโภคซึ่งจัดทำแล้ว ไขที่ได้จากสัตว์หรือพืช',
       'Animal or vegetable fats and oils and their cleavage products ; prepared edible fats ; animal or vegetable waxes'],
      dtype=object)

In [3]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [38]:
sentences2

array(['รองศาสตราจารย์ ชัชชาติ สิทธิพันธุ์', 'ธงไชย แมคอินไตย์',
       'บริษัท ซีพี ออลล์ จํากัด (มหาชน)',
       'CP ALL PUBLIC COMPANY LIMITED', 'บริษัท ปตท. จำกัด (มหาชน)',
       'PTT Public Company Limited '], dtype=object)

In [15]:
sentences1 = df['Query'].values
sentences2 = df['Key'].unique()

In [18]:
pd.DataFrame(sentences1)

,0
0,ข้าวโพด
1,Maize (corn).
2,ข้าว
3,Rice.
4,น้ำเลี้ยง (แซป) และสิ่งสกัดจากพืช สารจำพวกเพกต...
5,Vegetable saps and extracts; pectic substances...
6,น้ำมันมะพร้าว น้ำมันเนื้อในเมล็ดปาล์ม น้ำมันบา...
7,"Coconut (copra), palm kernel or babassu oil an..."
8,ไขมันและน้ำมันชนิดระเหยยากอื่น ๆ ที่ได้จากพืช ...
9,Other fixed vegetable fats and oils (including...


In [16]:
pd.DataFrame(sentences2)

,0
0,ธัญพืช
1,Cereals
2,ครั่ง รวมทั้งกัม เรซิน น้ำเลี้ยง(แซบ) และสิ่งส...
3,"Lac ; gum, resins and other vegetable saps and..."
4,ไขมันและน้ำมันที่ได้จากสัตว์หรือพืช และผลิตภัณ...
5,Animal or vegetable fats and oils and their cl...


In [19]:
#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score


for i in range(len(sentences1)):
    sort_index = cosine_scores[i].argsort(descending=True)
    for j in sort_index:
#         print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[j], cosine_scores[i][j]))
        print("{} \t\t {} \t\t Score: {:.4f}".format(i, j, cosine_scores[i][j]))
    print('\n')

0 		 0 		 Score: 0.4356
0 		 1 		 Score: 0.3033
0 		 4 		 Score: 0.0064
0 		 2 		 Score: 0.0026
0 		 3 		 Score: -0.0361
0 		 5 		 Score: -0.0476


1 		 1 		 Score: 0.6976
1 		 0 		 Score: 0.5082
1 		 2 		 Score: 0.2769
1 		 4 		 Score: 0.1190
1 		 3 		 Score: 0.0752
1 		 5 		 Score: -0.0382


2 		 0 		 Score: 0.4706
2 		 1 		 Score: 0.3362
2 		 2 		 Score: 0.0118
2 		 4 		 Score: -0.0074
2 		 3 		 Score: -0.0389
2 		 5 		 Score: -0.0564


3 		 1 		 Score: 0.6313
3 		 0 		 Score: 0.5102
3 		 2 		 Score: 0.2498
3 		 4 		 Score: 0.0948
3 		 3 		 Score: 0.0699
3 		 5 		 Score: -0.0420


4 		 2 		 Score: 0.5760
4 		 4 		 Score: 0.3594
4 		 3 		 Score: 0.3137
4 		 5 		 Score: 0.2168
4 		 0 		 Score: 0.1281
4 		 1 		 Score: 0.0061


5 		 3 		 Score: 0.5464
5 		 2 		 Score: 0.4232
5 		 5 		 Score: 0.4171
5 		 4 		 Score: 0.3770
5 		 0 		 Score: 0.1311
5 		 1 		 Score: 0.0209


6 		 4 		 Score: 0.5065
6 		 5 		 Score: 0.3169
6 		 2 		 Score: 0.1331
6 		 3 		 Score: 0.0725
6 		 0 		 Score: 0.05

In [65]:
cosine_scores[0].argsort(descending=True)

tensor([4, 1, 2, 0, 5, 3])

In [54]:
len(embeddings2)

6

In [15]:
emb1 = model.encode("ชัชชาติ")
emb2 = model.encode("รองศาสตราจารย์ ชัชชาติ สิทธิพันธุ์")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.3632]])


In [20]:
emb1 = model.encode("เบิร์ด ธงไชย")
emb2 = model.encode("ธงไชย แมคอินไตย์")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.5646]])


In [21]:
emb1 = model.encode("ชัชชาติ")
emb2 = model.encode("ธงไชย แมคอินไตย์")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.3990]])


In [33]:
textdistance.jaro_winkler.distance('เบิร์ด ธงไชย', 'รองศาสตราจารย์ ชัชชาติ สิทธิพันธุ์')

0.5522875816993464

In [36]:
textdistance.jaro_winkler.distance('เบิร์ด ธงไชย', 'เบิร์ด ธงไชยdddd')

0.050000000000000044

In [34]:
textdistance.jaro_winkler.distance('เบิร์ด ธงไชย', 'บริษัท')

0.5277777777777779

In [7]:
! pip install textdistance